### AVD894 – Course project and term paper
#### LunarLander-v2
##### By Team Reward Maximizers

In [149]:
# Define a grid of 5x5 (assuming discrete state space)
HEIGHT = 5
WIDTH = 5

# Define actions:
## 0 -> Do nothing
## 1 -> Fire left engine (go left)
## 2 -> Fire main engine (slows if fast, hovers if slow)
## 3 -> Fire right engine (go right)
### Always moves down under gravity, if unaffected by main engine thrust
# ACTIONS = [0, 1, 2, 3]
ACTIONS = {
    0: 'Nothing',
    1: 'Left',
    2: 'Main',
    3: 'Right'
}

# Define velocity
## 0 -> slow
## 1 -> fast
VELOCITY = [0, 1]

# x-coordinate of landing pad
LANDING_PAD_X = 2

# MDP Parameters
GAMMA = 0.99  # Discount factor
THETA = 0.0001  # Convergence threshold for Value Iteration

# Function to get states
## States are (x-coordinate, y-coordinate, velocity) U ('Landed', 'Crashed')
def get_states():
  states = []
  for x in range(WIDTH):
    for y in range(HEIGHT):
      for v in range(len(VELOCITY)):
        if y < 4:
          states.append((x,y,v))
  
  states.append('Landed')
  states.append('Crashed')
  return states

# Function to get next state and reward, given current state and action
## Equivalent to transition probability matrix and reward matrix
def get_next_state_and_reward(state, action):
  # Handle terminal conditions
  if state == 'Landed' or state == 'Crashed':
    return state, 0
      
  # Extract coordinates and velocity
  x, y, v = state

  # Initialise next states and reward
  x_next, y_next, v_next = x, y, v
  reward = 0

  # Handle different actions
  ## Always accelerate (velocity = 1) under gravity, except if main engine is used
  ## Reward of '-1' is any engine is used
  ## Don't allow movement is the lander is going out of the scene (grid)
  if action == 0: # do nothing
    y_next = y + 1
    v_next = 1

  elif action == 1: # left
    reward = -1
    y_next = y + 1
    v_next = 1
    if x > 0:
      x_next = x - 1

  elif action == 2: # main
    # slows if fast, hovers if slow
    reward = -1
    if v == 1:
      y_next = y + 1
      v_next = 0
    else:
      y_next = y
      v_next = 0
  
  elif action == 3: # right
    reward = -1
    y_next = y + 1
    v_next = 1
    if x < (WIDTH - 1):
      x_next = x + 1

  # Handle Final Conditions
  if y_next >= HEIGHT - 1:
    y_next = HEIGHT - 1 # Set to ground level
    if x_next == LANDING_PAD_X and v_next == 0: # Perfect slow landing
      return 'Landed', 100
    else:
      # Crashed or missed landing pad
      return 'Crashed', -100

  return (x_next, y_next, v_next), reward


In [150]:
# Funtio to solve the MDP using value iteration
def value_iteration():
    states = get_states()
    
    # Initialise V(s) = 0 for all states
    V = {s: 0 for s in states}
    
    # Set terminal state values which never change
    V['Landed'] = 100
    V['Crashed'] = -100

    iteration = 0
    while True:
        delta = 0
        V_new = V.copy()
        
        # Loop over all the states
        for s in states:
            if s == 'Landed' or s == 'Crashed':
                continue
            
            # Calculate Q(s,a) to get best action
            action_values = []
            for a in ACTIONS:
                next_s, reward = get_next_state_and_reward(s, a)
                
                # Bellman update equation:
                ## Q(s,a) = R(s,a,s') + gamma * V(s')
                q_value = reward + GAMMA * V[next_s]
                action_values.append(q_value)
            
            # V(s) = max[a](Q(s,a))
            best_s_value = max(action_values)
            
            # Calculate the change
            delta = max(delta, abs(best_s_value - V[s]))
            
            # Update the value for this state
            V_new[s] = best_s_value
        
        # Update the value function for the next iteration
        V = V_new
        iteration += 1

        # Check for convergence
        if delta < THETA:
            print(f"Value Iteration converged after {iteration} iterations.\n")
            break
            
    return V

In [151]:
# Policy Extraction
def extract_policy(V):
    policy = {}
    for s in get_states():
        if s == 'Landed' or s == 'Crashed':
            policy[s] = 'Done'
            continue
            
        # Find the best action
        best_action = -1
        best_q_value = -float('inf')
        
        for a in ACTIONS:
            next_s, reward = get_next_state_and_reward(s, a)
            q_value = reward + GAMMA * V[next_s]
            
            if q_value > best_q_value:
                best_q_value = q_value
                best_action = a
                
        policy[s] = ACTIONS[best_action]
        
    return policy

# Print the policy
def print_policy(policy, V):
    print("--- Optimal Policy (pi*) ---\n")
    
    for v in [0, 1]:
        print(f"Policy for {'slow' if v == 0 else 'fast'} velocity:\n")
        print(" y --------------")
        for y in (range(0, HEIGHT - 1)):
            row_str = f" {y} "
            for x in range(WIDTH):
                action = policy[(x, y, v)]
                # Display actions
                if action == 'Nothing': symbol = ' o '
                elif action == 'Left':  symbol = ' < '
                elif action == 'Main':  symbol = ' ^ '
                elif action == 'Right': symbol = ' > '
                else: symbol = ' ? '
                row_str += symbol
            print(row_str)
        print("    --------------")
        print("     0  1  2  3  4       x\n")

In [152]:
# Solve MDP using value iteration
optimal_V = value_iteration()

# Extract and print the optimal policy
optimal_policy = extract_policy(optimal_V)

print_policy(optimal_policy, optimal_V)

Value Iteration converged after 919 iterations.

--- Optimal Policy (pi*) ---

Policy for slow velocity:

 y --------------
 0  o  o  o  o  o 
 1  >  o  o  o  < 
 2  ^  >  o  <  ^ 
 3  ^  ^  ^  ^  ^ 
    --------------
     0  1  2  3  4       x

Policy for fast velocity:

 y --------------
 0  o  o  o  o  o 
 1  >  o  o  o  < 
 2  ^  >  o  <  ^ 
 3  o  o  ^  o  o 
    --------------
     0  1  2  3  4       x



In [153]:
# Print the value function
print("--- Optimal Value Function (V*) ---")
for s in (optimal_V):
    print(f"{str(s)}: {optimal_V[s]:.2f}")

--- Optimal Value Function (V*) ---
(0, 0, 0): 191.12
(0, 0, 1): 191.12
(0, 1, 0): 193.05
(0, 1, 1): 193.05
(0, 2, 0): -99.99
(0, 2, 1): -99.99
(0, 3, 0): -99.99
(0, 3, 1): -199.00
(1, 0, 0): 192.11
(1, 0, 1): 192.11
(1, 1, 0): 194.05
(1, 1, 1): 194.05
(1, 2, 0): 196.01
(1, 2, 1): 196.01
(1, 3, 0): -99.99
(1, 3, 1): -199.00
(2, 0, 0): 193.09
(2, 0, 1): 193.09
(2, 1, 0): 195.04
(2, 1, 1): 195.04
(2, 2, 0): 197.01
(2, 2, 1): 197.01
(2, 3, 0): -99.99
(2, 3, 1): 199.00
(3, 0, 0): 192.11
(3, 0, 1): 192.11
(3, 1, 0): 194.05
(3, 1, 1): 194.05
(3, 2, 0): 196.01
(3, 2, 1): 196.01
(3, 3, 0): -99.99
(3, 3, 1): -199.00
(4, 0, 0): 191.12
(4, 0, 1): 191.12
(4, 1, 0): 193.05
(4, 1, 1): 193.05
(4, 2, 0): -99.99
(4, 2, 1): -99.99
(4, 3, 0): -99.99
(4, 3, 1): -199.00
Landed: 100.00
Crashed: -100.00
